In [1]:
# https://jabsom.hawaii.edu/report-finds-hawai%CA%BBi-physician-shortage-exacerbated-by-the-pandemic/

# Step 1: Scraping 

#### referred to codes from: https://youtu.be/wCoTJdhRcQE 

### getting a list of primary care physicians in Hawaii

In [2]:
import pandas as pd
import requests
from selenium import webdriver as wb
from webdriver_manager.chrome import ChromeDriverManager

/Users/lisakim/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
webD = wb.Chrome(ChromeDriverManager().install())
webD.get('https://www.healthgrades.com/usearch?what=Primary%20Care&entityCode=PS305&searchType=PracticingSpecialty&where=Honolulu%2C%20HI&pt=21.304770%2C-157.857605&distances=100&pageNum=1&sort.provider=bestmatch&state=HI')
import time

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472
[WDM] - Driver [/Users/lisakim/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


In [4]:
listOfLinks=[]

i = 1 
while i <= 95:
    time.sleep(5)
    doctorInfoList = webD.find_elements_by_class_name("card-info__summary-info")
    for doctor in doctorInfoList:
        names = doctor.find_element_by_tag_name('h3').find_element_by_tag_name('a').get_property('href')
    
        listOfLinks.append(names)
 
    buttons = webD.find_elements_by_class_name('pRyBq')[-1]
    buttons.send_keys('\n')
        
    i += 1

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=91.0.4472.114)


In [ ]:
len(listOfLinks)

# Step 2: Getting individual info of each physician

In [6]:
from tqdm import tqdm

Address=[]
time.sleep(3)
for i in tqdm(listOfLinks):
    webD.get(i)
    name1 = webD.find_element_by_class_name('street-address').text
    name2 = webD.find_element_by_xpath("//span[@class='street-address']/following-sibling::span").text
    name3 = webD.find_element_by_xpath("//span[@class='street-address']/following-sibling::span/following-sibling::span").text
    name4 = webD.find_element_by_xpath("//span[@class='street-address']/following-sibling::span/following-sibling::span/following-sibling::span").text

    doctorInfo={'name1':name1,
              'name2':name2,
              'name3':name3,
              'name4':name4,
              'link':i}
    
    Address.append(doctorInfo)

webD.quit()    

100%|██████████| 2076/2076 [39:22<00:00,  1.14s/it] 


### saving Address (or the list of dictionaries) as a dataframe

In [7]:
# saving the list of dictionaries as a dataframe 
df = pd.DataFrame(Address)

In [8]:
# removing commas in df.name2
df.name2 = df.name2.astype(str)
df.name2 = df.name2.str.replace(',', '')

In [9]:
# making an address column
df['address'] = df[df.columns[0:4]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
df

,name1,name2,name3,name4,link,address
0,1188 Bishop St Ste 3208,Honolulu,HI,96813,https://www.healthgrades.com/physician/dr-edwa...,"1188 Bishop St Ste 3208,Honolulu,HI,96813"
1,1150 S King St Ste 302,Honolulu,HI,96814,https://www.healthgrades.com/physician/dr-nanc...,"1150 S King St Ste 302,Honolulu,HI,96814"
2,1100 Ward Ave,Honolulu,HI,96814,https://www.healthgrades.com/physician/dr-rand...,"1100 Ward Ave,Honolulu,HI,96814"
3,550 S Beretania St Ste 601,Honolulu,HI,96813,https://www.healthgrades.com/physician/dr-jess...,"550 S Beretania St Ste 601,Honolulu,HI,96813"
4,888 S King St,Honolulu,HI,96813,https://www.healthgrades.com/physician/dr-john...,"888 S King St,Honolulu,HI,96813"
...,...,...,...,...,...,...
2071,1190 Waianuenue Ave,Hilo,HI,96720,https://www.healthgrades.com/physician/dr-cole...,"1190 Waianuenue Ave,Hilo,HI,96720"
2072,1325 S Kihei Rd Ste 103,Kihei,HI,96753,https://www.healthgrades.com/physician/dr-meli...,"1325 S Kihei Rd Ste 103,Kihei,HI,96753"
2073,270 Dairy Rd Ste 239,Kahului,HI,96732,https://www.healthgrades.com/physician/dr-mary...,"270 Dairy Rd Ste 239,Kahului,HI,96732"
2074,2349 S Kihei Rd Ste D,Kihei,HI,96753,https://www.healthgrades.com/physician/dr-elme...,"2349 S Kihei Rd Ste D,Kihei,HI,96753"


In [10]:
df = df[df.name3 == "HI"]

In [11]:
df.shape

(1997, 6)

In [12]:
# saving as a csv 
df.to_csv('HawaiiPrimaryCarePhysician_List.csv', index=False)

# Step 3: Getting coordinates from addresses
#### got geocoordinates using Google Spreadsheet and 'Geocode by Awessome Table'
#### --> referred to https://www.youtube.com/watch?v=PX6IDvX6_Z8 

In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_colwidth", 200)

In [ ]:
df2 = pd.read_csv("HawaiiPhysicians_Coordinates.csv")

In [ ]:
df2.head(2) 

# Step 4: Mapping using Mapbox
#### referred to Mapbox's tutorial: https://docs.mapbox.com/help/tutorials/add-points-pt-2/